## GZ_Traffic_Tianchi_Session_1
- ZS167275
- 线上0.300成绩，最终第一赛季53名

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc

from scipy.stats import mode
import xgboost as xgb

/software/home/chenzh/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# 链路信息，包括长宽、链路等级
link_info = pd.read_table('data/gy_contest_link_info.txt',sep=';')
link_info = link_info.sort_values('link_ID')

In [4]:
training_data = pd.read_table(u'data/gy_contest_traveltime_training_data_second.txt',sep=';')
training_data.columns = ['link_ID', 'date', 'time_interval', 'travel_time']
print training_data.head()
print training_data.shape
training_data = pd.merge(training_data,link_info,on='link_ID')

               link_ID        date                              time_interval  \
0  4377906283422600514  2017-05-06  [2017-05-06 11:04:00,2017-05-06 11:06:00)   
1  3377906289434510514  2017-05-06  [2017-05-06 10:42:00,2017-05-06 10:44:00)   
2  3377906285934510514  2017-05-06  [2017-05-06 11:56:00,2017-05-06 11:58:00)   
3  3377906285934510514  2017-05-06  [2017-05-06 17:46:00,2017-05-06 17:48:00)   
4  3377906287934510514  2017-05-06  [2017-05-06 10:52:00,2017-05-06 10:54:00)   

   travel_time  
0          3.0  
1          1.0  
2         35.2  
3         26.2  
4         10.4  
(7705175, 4)


In [6]:
# 测试数据
testing_data = pd.read_table(u'data/gy_contest_result_template.txt',sep='#',header=None)
testing_data.columns = ['link_ID', 'date', 'time_interval', 'travel_time']
testing_data = pd.merge(testing_data,link_info,on='link_ID')
testing_data['travel_time'] = np.NaN
print testing_data.shape
# 所有数据
feature_date = pd.concat([training_data,testing_data],axis=0)

(118800, 7)


In [8]:
feature_date = feature_date.sort_values(['link_ID','time_interval'])
print feature_date
feature_date.to_csv('data/feature_data.csv',index=False)

                     link_ID        date  \
4316799  3377906280028510514  2017-03-01   
4317063  3377906280028510514  2017-03-01   
4316809  3377906280028510514  2017-03-01   
4316850  3377906280028510514  2017-03-01   
4316947  3377906280028510514  2017-03-01   
4316810  3377906280028510514  2017-03-01   
4316765  3377906280028510514  2017-03-01   
4317064  3377906280028510514  2017-03-01   
4317049  3377906280028510514  2017-03-01   
4316885  3377906280028510514  2017-03-01   
4317065  3377906280028510514  2017-03-01   
4317039  3377906280028510514  2017-03-01   
4316948  3377906280028510514  2017-03-01   
4316811  3377906280028510514  2017-03-01   
4317066  3377906280028510514  2017-03-01   
4316989  3377906280028510514  2017-03-01   
4316932  3377906280028510514  2017-03-01   
4316916  3377906280028510514  2017-03-01   
4316766  3377906280028510514  2017-03-01   
4316767  3377906280028510514  2017-03-01   
4317067  3377906280028510514  2017-03-01   
4316917  3377906280028510514  20

In [10]:
def AddBaseTimeFeature(df):
    df['time_interval_begin'] = pd.to_datetime(df['time_interval'].map(lambda x: x[1:20]))  # 取出时间序列
    df = df.drop(['date', 'time_interval'], axis=1)
    df['time_interval_month'] = df['time_interval_begin'].map(lambda x: x.strftime('%m'))  # 月份
    df['time_interval_day'] = df['time_interval_begin'].map(lambda x: x.day)  # 天数
    df['time_interval_begin_hour'] = df['time_interval_begin'].map(lambda x: x.strftime('%H'))  # 小时
    df['time_interval_minutes'] = df['time_interval_begin'].map(lambda x: x.strftime('%M'))  # 分钟
    # Monday=1, Sunday=7
    df['time_interval_week'] = df['time_interval_begin'].map(lambda x: x.weekday() + 1)  # 星期几
    return df

In [11]:
feature_data = pd.read_csv('data/feature_data.csv')
feature_data_date = AddBaseTimeFeature(feature_data)
print feature_data_date
feature_data_date.to_csv('data/feature_data.csv',index=False)

/software/home/chenzh/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


                     link_ID  travel_time  length  width  link_class  \
0        3377906280028510514          5.4      48      3           1   
1        3377906280028510514          5.4      48      3           1   
2        3377906280028510514          5.4      48      3           1   
3        3377906280028510514          4.4      48      3           1   
4        3377906280028510514          4.4      48      3           1   
5        3377906280028510514          4.3      48      3           1   
6        3377906280028510514          4.3      48      3           1   
7        3377906280028510514          4.5      48      3           1   
8        3377906280028510514          4.5      48      3           1   
9        3377906280028510514          4.5      48      3           1   
10       3377906280028510514          4.5      48      3           1   
11       3377906280028510514          4.2      48      3           1   
12       3377906280028510514          4.2      48      3        

In [12]:
# test
feature_data = pd.read_csv('data/feature_data.csv')
test = feature_data.loc[(feature_data.time_interval_month == 6)&(feature_data.time_interval_begin_hour==8),: ]
test.to_csv('data/test.csv',index=False)

In [14]:
def mode_function(df):
    counts = mode(df)
    return counts[0][0]

In [15]:
feature_data = pd.read_csv('data/feature_data.csv')
feature_data['link_ID'] = feature_data['link_ID'].astype(str)
# link_info_count = pd.read_csv('./pre_data/link_info_count.csv')
# link_info_count['link_ID'] = link_info_count['link_ID'].astype(str)
# feature_data = pd.merge(feature_data,link_info_count,on='link_ID',how='left')
# link_class = pd.get_dummies(feature_data['link_class'],prefix='link_class')
# int_count_onehot = pd.get_dummies(feature_data['in_count_'],prefix='in_count')
# out_count_onehot = pd.get_dummies(feature_data['out_count_'],prefix='out_count')
week = pd.get_dummies(feature_data['time_interval_week'],prefix='week')
# time_interval_minutes = pd.get_dummies(feature_data['time_interval_minutes'],prefix='time_interval_minutes')
# day = pd.get_dummies(feature_data['time_interval_day'],prefix='day')
feature_data.drop(['time_interval_week','link_class'],inplace=True,axis=1)
# linkId = pd.get_dummies(feature_data['link_ID'],prefix='link_id')
feature_data = pd.concat([feature_data,week],axis=1)
print feature_data.head()

               link_ID  travel_time  length  width  time_interval_begin  \
0  3377906280028510514          5.4      48      3  2017-03-01 00:32:00   
1  3377906280028510514          5.4      48      3  2017-03-01 00:34:00   
2  3377906280028510514          5.4      48      3  2017-03-01 00:36:00   
3  3377906280028510514          4.4      48      3  2017-03-01 00:46:00   
4  3377906280028510514          4.4      48      3  2017-03-01 00:48:00   

   time_interval_month  time_interval_day  time_interval_begin_hour  \
0                    3                  1                         0   
1                    3                  1                         0   
2                    3                  1                         0   
3                    3                  1                         0   
4                    3                  1                         0   

   time_interval_minutes  week_1  week_2  week_3  week_4  week_5  week_6  \
0                     32       0       0      

In [26]:
# 4月8时
train = feature_data.loc[(feature_data.time_interval_month == 4)&(feature_data.time_interval_begin_hour==8),: ]
for i in [58,48,38,28,18,8,0]:
    # 4，7时数据，min大于等于i
    tmp = feature_data.loc[(feature_data.time_interval_month == 4)&(feature_data.time_interval_begin_hour == 7)&(feature_data.time_interval_minutes >= i),:]
    tmp = tmp.groupby(['link_ID', 'time_interval_day'])[
            'travel_time'].agg([('mean_%d' % (i), np.mean), ('median_%d' % (i), np.median),
                                ('mode_%d' % (i), mode_function), ('std_%d' % (i), np.std), ('max_%d' % (i), np.max),('min_%d' % (i), np.min)]).reset_index()
    tmp['std_%d' % (i)] = tmp['std_%d' % (i)].fillna(0)
    train = pd.merge(train,tmp,on=['link_ID','time_interval_day'],how='left')

In [28]:
# 4月18时
train_ = feature_data.loc[(feature_data.time_interval_month == 4)&(feature_data.time_interval_begin_hour==18),: ]
for i in [58,48,38,28,18,8,0]:
    tmp = feature_data.loc[(feature_data.time_interval_month == 4)&(feature_data.time_interval_begin_hour == 17)&(feature_data.time_interval_minutes >= i),:]
    tmp = tmp.groupby(['link_ID', 'time_interval_day'])[
            'travel_time'].agg([('mean_%d' % (i), np.mean), ('median_%d' % (i), np.median),
                                ('mode_%d' % (i), mode_function), ('std_%d' % (i), np.std), ('max_%d' % (i), np.max),('min_%d' % (i), np.min)]).reset_index()
    tmp['std_%d' % (i)] = tmp['std_%d' % (i)].fillna(0)
    train_ = pd.merge(train_,tmp,on=['link_ID','time_interval_day'],how='left')

In [29]:
# 5月18时
train__ = feature_data.loc[(feature_data.time_interval_month == 5)&(feature_data.time_interval_begin_hour==18),: ]
for i in [58,48,38,28,18,8,0]:
    tmp = feature_data.loc[(feature_data.time_interval_month == 5)&(feature_data.time_interval_begin_hour == 17)&(feature_data.time_interval_minutes >= i),:]
    tmp = tmp.groupby(['link_ID', 'time_interval_day'])[
            'travel_time'].agg([('mean_%d' % (i), np.mean), ('median_%d' % (i), np.median),
                                ('mode_%d' % (i), mode_function), ('std_%d' % (i), np.std), ('max_%d' % (i), np.max),('min_%d' % (i), np.min)]).reset_index()
    tmp['std_%d' % (i)] = tmp['std_%d' % (i)].fillna(0)
    train__ = pd.merge(train__,tmp,on=['link_ID','time_interval_day'],how='left')
# 5月9时
train___ = feature_data.loc[(feature_data.time_interval_month == 5)&(feature_data.time_interval_begin_hour==9),: ]
for i in [58,48,38,28,18,8,0]:
    tmp = feature_data.loc[(feature_data.time_interval_month == 5)&(feature_data.time_interval_begin_hour == 8)&(feature_data.time_interval_minutes >= i),:]
    tmp = tmp.groupby(['link_ID', 'time_interval_day'])[
            'travel_time'].agg([('mean_%d' % (i), np.mean), ('median_%d' % (i), np.median),
                                ('mode_%d' % (i), mode_function), ('std_%d' % (i), np.std), ('max_%d' % (i), np.max),('min_%d' % (i), np.min)]).reset_index()
    tmp['std_%d' % (i)] = tmp['std_%d' % (i)].fillna(0)
    train___ = pd.merge(train___,tmp,on=['link_ID','time_interval_day'],how='left')
# 5月19时
train____ = feature_data.loc[(feature_data.time_interval_month == 5)&(feature_data.time_interval_begin_hour==19),: ]
for i in [58,48,38,28,18,8,0]:
    tmp = feature_data.loc[(feature_data.time_interval_month == 5)&(feature_data.time_interval_begin_hour == 18)&(feature_data.time_interval_minutes >= i),:]
    tmp = tmp.groupby(['link_ID', 'time_interval_day'])[
            'travel_time'].agg([('mean_%d' % (i), np.mean), ('median_%d' % (i), np.median),
                                ('mode_%d' % (i), mode_function), ('std_%d' % (i), np.std), ('max_%d' % (i), np.max),('min_%d' % (i), np.min)]).reset_index()
    tmp['std_%d' % (i)] = tmp['std_%d' % (i)].fillna(0)
    train____ = pd.merge(train____,tmp,on=['link_ID','time_interval_day'],how='left')
# 5月7时
train_____ = feature_data.loc[(feature_data.time_interval_month == 5)&(feature_data.time_interval_begin_hour==7),: ]
for i in [58,48,38,28,18,8,0]:
    tmp = feature_data.loc[(feature_data.time_interval_month == 5)&(feature_data.time_interval_begin_hour == 6)&(feature_data.time_interval_minutes >= i),:]
    tmp = tmp.groupby(['link_ID', 'time_interval_day'])[
            'travel_time'].agg([('mean_%d' % (i), np.mean), ('median_%d' % (i), np.median),
                                ('mode_%d' % (i), mode_function), ('std_%d' % (i), np.std), ('max_%d' % (i), np.max),('min_%d' % (i), np.min)]).reset_index()
    tmp['std_%d' % (i)] = tmp['std_%d' % (i)].fillna(0)
    train_____ = pd.merge(train_____,tmp,on=['link_ID','time_interval_day'],how='left')

In [30]:
train = pd.concat([train,train_,train_____,train___,train__,train____],axis=0)

In [32]:
# 3月
train_history = feature_data.loc[(feature_data.time_interval_month == 3),: ]
train_history = train_history.groupby(['link_ID', 'time_interval_minutes'])[
            'travel_time'].agg([('mean_m', np.mean), ('median_m', np.median),
                                ('mode_m', mode_function), ('std_m', np.std), ('max_m', np.max),('min_m', np.min)]).reset_index()

In [33]:
train = pd.merge(train,train_history,on=['link_ID','time_interval_minutes'],how='left')

In [34]:
train['speed_mode'] = train['length']  / train['mode_m']
train['speed_median'] = train['length']  / train['median_m']

In [35]:
train['mean_std'] = train['mean_m']  / train['std_m']
train['max_min_distance'] = train['max_m'] - train['min_m']

In [36]:
train_8 = feature_data.loc[(feature_data.time_interval_month == 3)&(feature_data.time_interval_begin_hour == 8),: ]
train_8 = train_8.groupby(['link_ID', 'time_interval_minutes'])[
            'travel_time'].agg([('median_8_', np.median),('mode_8_', mode_function)]).reset_index()

train = pd.merge(train,train_8,on=['link_ID','time_interval_minutes'],how='left')

print train.shape

(690029, 70)


In [37]:
train_label = np.log1p(train.pop('travel_time'))

In [38]:
test = feature_data.loc[(feature_data.time_interval_month == 5)&(feature_data.time_interval_begin_hour==8),: ]

In [39]:
for i in [58,48,38,28,18,8,0]:
    tmp = feature_data.loc[(feature_data.time_interval_month == 5)&(feature_data.time_interval_begin_hour == 7)&(feature_data.time_interval_minutes >= i),:]
    tmp = tmp.groupby(['link_ID', 'time_interval_day'])[
            'travel_time'].agg([('mean_%d' % (i), np.mean), ('median_%d' % (i), np.median),
                                ('mode_%d' % (i), mode_function), ('std_%d' % (i), np.std), ('max_%d' % (i), np.max),('min_%d' % (i), np.min)]).reset_index()
    # tmp['std_%d' % (i)] = tmp['std_%d' % (i)].fillna(0)
    test = pd.merge(test,tmp,on=['link_ID','time_interval_day'],how='left')


test_history = feature_data.loc[(feature_data.time_interval_month == 4),: ]
test_history = test_history.groupby(['link_ID', 'time_interval_minutes'])[
            'travel_time'].agg([('mean_m', np.mean), ('median_m', np.median),
                                ('mode_m', mode_function), ('std_m', np.std), ('max_m', np.max),('min_m', np.min)]).reset_index()
# test_history['median_mode'] = 0.5 * test_history['mode_m'] + 0.5 * test_history['median_m']
test = pd.merge(test,test_history,on=['link_ID','time_interval_minutes'],how='left')

# test['speed_max'] = test['length']  / test['min_m']
# test['speed_min'] = test['length']  / test['max_m']
test['speed_mode'] = test['length']  / test['mode_m']
test['speed_median'] = test['length']  / test['median_m']

# test['120_speed'] = test['length']  / 120.0
test['mean_std'] = test['mean_m']  / test['std_m']
test['max_min_distance'] = test['max_m'] - test['min_m']

test_8 = feature_data.loc[(feature_data.time_interval_month == 4)&(feature_data.time_interval_begin_hour == 8),: ]
test_8 = test_8.groupby(['link_ID', 'time_interval_minutes'])[
            'travel_time'].agg([('median_8_', np.median),('mode_8_', mode_function)]).reset_index()

test = pd.merge(test,test_8,on=['link_ID','time_interval_minutes'],how='left')

print test.head()

test_label = np.log1p(test.pop('travel_time'))

train.drop(['time_interval_begin_hour','time_interval_month','time_interval_begin','std_m'],inplace=True,axis=1)
test.drop(['time_interval_begin_hour','time_interval_month','time_interval_begin','std_m'],inplace=True,axis=1)

               link_ID  travel_time  length  width  time_interval_begin  \
0  3377906280028510514          3.9      48      3  2017-05-01 08:08:00   
1  3377906280028510514          3.9      48      3  2017-05-01 08:10:00   
2  3377906280028510514          3.9      48      3  2017-05-01 08:12:00   
3  3377906280028510514          3.9      48      3  2017-05-01 08:14:00   
4  3377906280028510514          4.0      48      3  2017-05-01 08:20:00   

   time_interval_month  time_interval_day  time_interval_begin_hour  \
0                    5                  1                         8   
1                    5                  1                         8   
2                    5                  1                         8   
3                    5                  1                         8   
4                    5                  1                         8   

   time_interval_minutes  week_1   ...     mode_m     std_m  max_m  min_m  \
0                      8       1   ...       

In [41]:
def mape_object(y,d):

    g=1.0*np.sign(y-d)/d
    h=1.0/d
    return -g,h

# 评价函数
def mape(y,d):
    c=d.get_label()
    result=np.sum(np.abs(y-c)/c)/len(c)
    return "mape",result

# 评价函数ln形式
def mape_ln(y,d):
    c=d.get_label()
    result=np.sum(np.abs(np.expm1(y)-np.abs(np.expm1(c)))/np.abs(np.expm1(c)))/len(c)
    return "mape",result


In [42]:
xlf = xgb.XGBRegressor(max_depth=11,
                       learning_rate=0.01,
                       n_estimators=1000,
                       silent=True,
                       objective=mape_object,
                       gamma=0,
                       min_child_weight=5,
                       max_delta_step=0,
                       subsample=0.8,
                       colsample_bytree=0.8,
                       colsample_bylevel=1,
                       reg_alpha=1e0,
                       reg_lambda=0,
                       scale_pos_weight=1,
                       seed=9,
                       missing=None)


xlf.fit(train.values, train_label.values, eval_metric=mape_ln, verbose=True, eval_set=[(test.values, test_label.values)],early_stopping_rounds=2)
print xlf.get_params()

[0]	validation_0-mape:0.882369
Will train until validation_0-mape hasn't improved in 2 rounds.
[1]	validation_0-mape:0.879446
[2]	validation_0-mape:0.876487
Stopping. Best iteration:
[0]	validation_0-mape:0.882369

{'reg_alpha': 1.0, 'colsample_bytree': 0.8, 'silent': True, 'colsample_bylevel': 1, 'scale_pos_weight': 1, 'learning_rate': 0.01, 'missing': None, 'max_delta_step': 0, 'nthread': -1, 'base_score': 0.5, 'n_estimators': 1000, 'subsample': 0.8, 'reg_lambda': 0, 'seed': 9, 'min_child_weight': 5, 'objective': <function mape_object at 0x7fd841e46488>, 'max_depth': 11, 'gamma': 0}


In [43]:
sub = feature_data.loc[(feature_data.time_interval_month == 6)&(feature_data.time_interval_begin_hour==8),: ]
for i in [58,48,38,28,18,8,0]:
    tmp = feature_data.loc[(feature_data.time_interval_month == 6)&(feature_data.time_interval_begin_hour == 7)&(feature_data.time_interval_minutes >= i),:]
    tmp = tmp.groupby(['link_ID', 'time_interval_day'])[
            'travel_time'].agg([('mean_%d' % (i), np.mean), ('median_%d' % (i), np.median),
                                ('mode_%d' % (i), mode_function), ('std_%d' % (i), np.std), ('max_%d' % (i), np.max),('min_%d' % (i), np.min)]).reset_index()
    tmp['std_%d' % (i)] = tmp['std_%d' % (i)].fillna(0)
    sub = pd.merge(sub,tmp,on=['link_ID','time_interval_day'],how='left')

sub_history = feature_data.loc[(feature_data.time_interval_month == 5),: ]
sub_history = sub_history.groupby(['link_ID', 'time_interval_minutes'])[
            'travel_time'].agg([('mean_m', np.mean), ('median_m', np.median),
                                ('mode_m', mode_function), ('std_m', np.std), ('max_m', np.max),('min_m', np.min)]).reset_index()
# sub_history['median_mode'] = 0.5 * sub_history['mode_m'] + 0.5 * sub_history['median_m']

sub = pd.merge(sub,sub_history,on=['link_ID','time_interval_minutes'],how='left')
# sub['speed_max'] = sub['length'] / sub['min_m']
# sub['speed_min'] = sub['length'] / sub['max_m']
sub['speed_mode'] = sub['length'] / sub['mode_m']
sub['speed_median'] = sub['length'] / sub['median_m']

# sub['120_speed'] = sub['length'] / 120.0

sub['mean_std'] = sub['mean_m']  / sub['std_m']
sub['max_min_distance'] = sub['max_m'] - sub['min_m']

sub_history_8 = feature_data.loc[(feature_data.time_interval_month == 5)&(feature_data.time_interval_begin_hour == 8),: ]
sub_history_8 = sub_history_8.groupby(['link_ID', 'time_interval_minutes'])[
            'travel_time'].agg([('median_8_', np.median),('mode_8_', mode_function)]).reset_index()

sub = pd.merge(sub,sub_history_8,on=['link_ID','time_interval_minutes'],how='left')

print sub.head()

sub_label = np.log1p(sub.pop('travel_time'))

sub.drop(['time_interval_begin_hour','time_interval_month','time_interval_begin','std_m'],inplace=True,axis=1)

result = xlf.predict(sub.values)

travel_time = pd.DataFrame({'travel_time':list(result)})
sub_demo = pd.read_table(u'data/gy_contest_result_template.txt',header=None,sep='#')

sub_demo.columns = ['link_ID','date','time_interval','travel_time']
sub_demo = sub_demo.sort_values(['link_ID','time_interval']).reset_index()
del sub_demo['index']

del sub_demo['travel_time']
tt = pd.concat([sub_demo,travel_time],axis=1)
# tt = tt.fillna(0)
tt['travel_time'] = np.round(np.expm1(tt['travel_time']),6)
tt[['link_ID','date','time_interval','travel_time']].to_csv('./mapodoufu_2017-08-12.txt',sep='#',index=False,header=False)
print tt[['link_ID','date','time_interval','travel_time']].shape
print tt[['link_ID','date','time_interval','travel_time']].isnull().sum()

               link_ID  travel_time  length  width  time_interval_begin  \
0  3377906280028510514          NaN      48      3  2017-06-01 08:00:00   
1  3377906280028510514          NaN      48      3  2017-06-01 08:02:00   
2  3377906280028510514          NaN      48      3  2017-06-01 08:04:00   
3  3377906280028510514          NaN      48      3  2017-06-01 08:06:00   
4  3377906280028510514          NaN      48      3  2017-06-01 08:08:00   

   time_interval_month  time_interval_day  time_interval_begin_hour  \
0                    6                  1                         8   
1                    6                  1                         8   
2                    6                  1                         8   
3                    6                  1                         8   
4                    6                  1                         8   

   time_interval_minutes  week_1   ...     mode_m     std_m  max_m  min_m  \
0                      0       0   ...       